# 네이버 영화리뷰 감정분석 with Hugging Face KoELECTRA and Data Augmentation(KNU 감성사전 + 스팀 리뷰)
참고 소스 출처(링크) : https://github.com/monologg/KoELECTRA<br>
https://blog.naver.com/horajjan/221739630055<br>
https://bab2min.tistory.com/657

# 준비
라이브러리, 파라미터 세팅

In [1]:
import tensorflow as tf
import torch

from transformers import ElectraTokenizer
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import os

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TEST_SIZE = 0.2
RANDOM_SEED = 42
EPOCHS = 4  # 3

In [3]:
# 디바이스 설정 확인
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 4 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [4]:
# GPU 할당 변경하기
GPU_NUM = 3 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

Current cuda device  3


# 데이터 로드

In [5]:
# 학습 데이터 로드
train = pd.read_csv(DATA_IN_PATH + 'ratings_train.txt', sep='\t')
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
# 검증(Original Test Dataset) 데이터 로드
dev = pd.read_csv(DATA_IN_PATH + 'ratings_test.txt', sep='\t')
dev.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [7]:
# 테스트(캐글) 데이터 로드
test = pd.read_csv(DATA_IN_PATH + 'ko_data.csv', encoding = 'cp949')
test.columns = ['id','document']  # 전처리 일괄 수행을 위해 컬럼명 변경(학습 데이터셋과 동일하게)
test.head()

,id,document
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


In [8]:
train.drop('id', axis=1, inplace=True)
#dev.drop('id', axis=1, inplace=True)

### 추가 데이터 로딩 (KNU 한국어 감성사전)
https://github.com/park1200656/KnuSentiLex

본 한국어 감성사전은 다음과 같은 소스로부터 통합되어 개발되었음<br>
(1) 국립국어원 표준국어대사전의 뜻풀이(glosses) 분석을 통한 긍부정 추출(이 방법을 통해 대부분의 긍부정어 추출)<br>
(2) 김은영(2004)의 긍부정어 목록<br>
(3) SentiWordNet 및 SenticNet-5.0에서 주로 사용되는 긍부정어 번역<br>
(4) 최근 온라인에서 많이 사용되는 축약어 및 긍부정 이모티콘 목록<br>

총 14,843개의 1-gram, 2-gram, 관용구, 문형, 축약어, 이모티콘 등에 대한 긍정, 중립, 부정 판별 및 정도(degree)값 계산

In [9]:
# 추가 데이터(naver_shopping.txt) 로딩 (Naver Shopping Review)
train_plus_df1 = pd.read_csv(DATA_IN_PATH + 'SentiWordDict/' + 'SentiWord_Dict.txt', sep='\t', header = None)  # 데이터프레임 로드
train_plus_df1.columns = ['document','label']  # 컬럼명 부여
train_plus_df1.head()

,document,label
0,(-;,1.0
1,(;_;),-1.0
2,(^^),1.0
3,(^-^),1.0
4,(^^*,1.0


In [10]:
# 전체 데이터 개수
len(train_plus_df1)

14855

In [11]:
# 레이블 현황 확인
train_plus_df1['label'].value_counts()

-1.0    5030
-2.0    4799
 2.0    2603
 1.0    2268
 0.0     154
Name: label, dtype: int64

In [12]:
# '0.0'(중립 감정) 레이블 제거
idx_neutral = train_plus_df1[train_plus_df1['label'] == 0.0].index
train_plus_df1 = train_plus_df1.drop(idx_neutral)
len(train_plus_df1)

14701

In [13]:
# 레이블 값 변경
train_plus_df1.label[train_plus_df1.label == -1.0] = 0  # -1.0 => 부정 0
train_plus_df1.label[train_plus_df1.label == -2.0] = 0  # -2.0 => 부정 0
train_plus_df1.label[train_plus_df1.label == 1.0] = 1  # 1.0 => 긍정 1
train_plus_df1.label[train_plus_df1.label == 2.0] = 1  # 2.0 => 긍정 1
train_plus_df1['label'].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

0.0    9829
1.0    4871
Name: label, dtype: int64

In [14]:
# null 값 여부 확인
print(train_plus_df1['label'].isnull().sum())

1


In [15]:
# null 값 row 제거
train_plus_df1.dropna(axis=0, inplace = True)

In [16]:
train_plus_df1.head()

,document,label
0,(-;,1.0
1,(;_;),0.0
2,(^^),1.0
3,(^-^),1.0
4,(^^*,1.0


In [17]:
print(train_plus_df1.dtypes)

document     object
label       float64
dtype: object


In [18]:
# 레이블 데이터 타입 변환(float64->int64)
train_plus_df1 = train_plus_df1.astype({'label': 'int64'})

In [19]:
print(train_plus_df1.dtypes)

document    object
label        int64
dtype: object


In [20]:
plus_result_tmp = pd.concat([train,train_plus_df1], axis=0, ignore_index=True)
train = plus_result_tmp.copy()
train

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
164695,오류,0
164696,의혹,0
164697,내팽개치다,0
164698,횡령,0


### 추가 데이터 로딩 2 (Steam Review)
https://github.com/bab2min/corpus/tree/master/sentiment

게임 유통 서비스인 Steam의 각종 게임에 달린 한국어 리뷰를 수집한 것.<br>
게임 커뮤니티 특성 상 비속어 및 은어가 많이 사용된 것이 특징이다.<br>
데이터는 탭으로 분리되어 있으며, 첫번째 필드에는 긍/부정(1=긍정, 0=부정), 두번째 필드에는 리뷰 텍스트가 위치한다.<br>
긍정과 부정의 비율이 1:1에 가깝도록 샘플링되었다.

In [21]:
# 추가 데이터(steam.txt) 로딩 (Steam Review)
train_plus_df2 = pd.read_csv(DATA_IN_PATH + 'steam/' + 'steam.txt', sep='\t', header = None)  # 데이터프레임 로드
train_plus_df2.columns = ['label', 'document']  # 컬럼명 부여
train_plus_df2 = train_plus_df2[['document','label']]  # 컬럼 순서 바꾸기

In [22]:
train_plus_df2.head()

,document,label
0,노래가 너무 적음,0
1,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다.",0
2,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지,1
3,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??,1
4,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요,1


In [23]:
plus_result_tmp2 = pd.concat([train,train_plus_df2], axis=0, ignore_index=True)
train = plus_result_tmp2.copy()
train

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
264695,한글화해주면 10개산다,0
264696,개쌉노잼 ㅋㅋ,0
264697,노잼이네요... 30분하고 지웠어요...,0
264698,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.,1


In [24]:
# 테이터 shape 확인
print(train.shape)
print(dev.shape)
print(test.shape)

(264700, 2)
(50000, 3)
(11187, 2)


In [25]:
# null 값 공백처리
train = train.fillna(' ')
dev = dev.fillna(' ')
test = test.fillna(' ')

In [26]:
# 데이터 타입 확인
print(train.dtypes)
print(dev.dtypes)
print(test.dtypes)

document    object
label        int64
dtype: object
id           int64
document    object
label        int64
dtype: object
id           int64
document    object
dtype: object


# 데이터 전처리

In [27]:
MAX_LEN = 128

def getInputs(dataset):
    data = dataset.copy(deep=True)
    
    if 'document' in data.columns:
        sentences = data['document']
    else:
        sentences = data['Sentence']
    sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
    
    tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", do_lower_case=False)
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    
    return input_ids, attention_masks

In [28]:
def getIndex(dataset):
    data = dataset.copy(deep = True)
    input_index = data.index.tolist()
    return torch.tensor(input_index)

In [29]:
# 훈련셋,검증셋 분리 전 학습 데이터셋 라벨 추출
labels = train['label'].values
# 검증(Original Test Dataset) 데이터셋 라벨 추출
dev_labels = dev['label'].values

ratings_inputs, ratings_masks = getInputs(train)
test_inputs, test_masks = getInputs(test)
dev_inputs, dev_masks = getInputs(dev)

In [30]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(ratings_inputs, labels, random_state=RANDOM_SEED, test_size=TEST_SIZE)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(ratings_masks, ratings_inputs, random_state=RANDOM_SEED, test_size=TEST_SIZE)

In [31]:
# 학습셋 텐서 변환(검증 포함)
# 학습/검증/테스트 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

# 테스트셋(Kaggle Dataset) 텐서 변환
test_index = getIndex(test)
test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)

# 검증용(Original Test Dataset) 텐서 변환
dev_inputs = torch.tensor(dev_inputs)
dev_labels = torch.tensor(dev_labels)
dev_masks = torch.tensor(dev_masks)

In [32]:
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 테스트셋(Kaggle Dataset)
test_data = TensorDataset(test_index, test_inputs, test_masks)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 검증용(Original Test Dataset)
dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels)
dev_sampler = RandomSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

# 모델 생성

In [33]:
# ELECTRA 모델 생성 (koelectra-base-v3-discriminator)
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels = 2)
model.cuda()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [34]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = EPOCHS

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 모델 학습

In [35]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [36]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = RANDOM_SEED
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로직과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  6,618.    Elapsed: 0:02:10.
  Batch 1,000  of  6,618.    Elapsed: 0:04:20.
  Batch 1,500  of  6,618.    Elapsed: 0:06:30.
  Batch 2,000  of  6,618.    Elapsed: 0:08:39.
  Batch 2,500  of  6,618.    Elapsed: 0:10:49.
  Batch 3,000  of  6,618.    Elapsed: 0:12:59.
  Batch 3,500  of  6,618.    Elapsed: 0:15:08.
  Batch 4,000  of  6,618.    Elapsed: 0:17:18.
  Batch 4,500  of  6,618.    Elapsed: 0:19:28.
  Batch 5,000  of  6,618.    Elapsed: 0:21:38.
  Batch 5,500  of  6,618.    Elapsed: 0:23:48.
  Batch 6,000  of  6,618.    Elapsed: 0:26:00.
  Batch 6,500  of  6,618.    Elapsed: 0:28:11.

  Average training loss: 0.32033
  Training epcoh took: 0:28:42

Running Validation...
  Accuracy: 0.87905
  Validation took: 0:02:25

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  6,618.    Elapsed: 0:02:12.
  Batch 1,000  of  6,618.    Elapsed: 0:04:24.
  Batch 1,500  of  6,618.    Elapsed: 0:06:36.
  Batch 2,000  of  6,618.  

# 검증 데이터셋(Original Test Dataset) 평가

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_steps, nb_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(dev_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dev_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_accuracy
    nb_steps += 1

print("")
print("Accuracy: {0:.5f}".format(eval_accuracy/nb_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

# (참고) 제출 파일 생성

In [ ]:
tmp_test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=1)
test_result = test.copy(deep = True)
test_result['Predicted'] = 'default'
classes = [0, 1]

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(tmp_test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_index, b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    idx = b_index.item()
    test_result['Predicted'][idx] = classes[np.argmax(logits)]
    
    nb_eval_steps += 1

print("")
print("Test took: {:}".format(format_time(time.time() - t0)))

In [ ]:
test_result.head()

In [ ]:
# 모델 산출물(데이터프레임) 수정
test_result.drop('document', axis=1, inplace=True)
test_result = test_result.rename({'id':'Id'}, axis='columns')
test_result

In [ ]:
# 해당 경로가 없으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# csv파일 생성
test_result.to_csv(DATA_OUT_PATH + "NSMC_KoELECTRA_SentiWordDict_Steam.csv", index = False)  # 앙상블 조합 및 캐글 제출

### 캐글 제출 결과
**[2020.12.11]**<br>
epoch 4 => 0.89325 (best)<br>
epoch 3 => 0.88932

---